In [16]:
import ase.io.vasp
import os
import siman
from siman.calc_manage import smart_structure_read

In [17]:
cell = ase.io.vasp.read_vasp("nio_Jcoupl/POSCAR")
ase.io.vasp.write_vasp('nio_Jcoupl/POSCAR_2', cell * (2,2,2), label='222supercell',direct=True,sort=True)

In [18]:
def count_nn(path_to_poscar, magnetic_atoms):
    """
    calculated the number of nearest neighbors,
    to fit into the Heisenberg model.
    Get a path to POSCAR structure as an input, 
    To avoid errors one should use prettified POSCAR,
    use poscar_prettifier() function first.
    
    Args:
        poscar_path   (str) - path to the POSCAR file
        magnetic_atoms(list) - two type of atoms to be treated as a magnetic
                            with an opposite spins (up/down).
                            your POSCAR should contain this two types of atoms.
    Returns:
        dict(distance : number_of_neibours) 
    """
    if not os.path.exists(path_to_poscar):
        print(f'File {path_to_poscar} does not exist!')
        return None
    st = smart_structure_read(path_to_poscar)
    st = st.replic([6, 6, 6])
    out = st.nn(i = 1, n = 50, silent=1)
    
    a = list(zip(out['el'][1:], out['dist'][1:]))
    unique_dist = set(round(i[1], 3) for i in a if i[0] in magnetic_atoms) #collect all the unique distances
    magnetic_dist_lst = [(el, round(dist, 3)) for el, dist in a if el in magnetic_atoms]

    dist_neighbNum = {_: 0 for _ in unique_dist} #key- distane, value list of 
                                                #neighbours at 1st 2nd 3d coordination spheres

    for dist in unique_dist:
        for el, distance in magnetic_dist_lst:
            if dist == distance:
                if el == magnetic_atoms[0]:
                    dist_neighbNum[dist] += 1
                elif el == magnetic_atoms[1]:
                    dist_neighbNum[dist] -= 1
    return dist_neighbNum

In [21]:
st = smart_structure_read('nio_Jcoupl/POSCAR_2')
st = st.replic([6, 6, 6])
out = st.nn(i = 1, n = 50, silent=1)

In [20]:
count_nn('nio_Jcoupl/POSCAR_2', magnetic_atoms=['Ni','Co'])

{2.949: -4, 4.17: -6}